In [2]:
%pip install ijson

     ---------------------------------------- 48.2/48.2 kB 1.2 MB/s eta 0:00:00


In [8]:
import ijson
import json
import os

In [4]:
# Step 1: Define Top Jazz Musicians
top_jazz_musicians = {
    "Frank Sinatra",
    "Norah Jones",
    "Ella Fitzgerald",
    "Nat King Cole",
    "Louis Armstrong",
    
}

In [5]:
# Helper function to process and filter a chunk
def process_chunk(chunk):
    filtered_playlists = []

    for playlist in chunk:
        filtered_tracks = []
        for track in playlist['tracks']:
            if track['artist_name'] in top_jazz_musicians:
                filtered_tracks.append(track)
        if filtered_tracks:
            filtered_playlist = {
                "name": playlist['name'],
                "collaborative": playlist['collaborative'],
                "pid": playlist['pid'],
                "modified_at": playlist['modified_at'],
                "num_tracks": len(filtered_tracks),
                "num_albums": len(set(track['album_uri'] for track in filtered_tracks)),
                "num_followers": playlist['num_followers'],
                "tracks": filtered_tracks
            }
            filtered_playlists.append(filtered_playlist)
    return filtered_playlists

In [6]:
# Step 2: Iterate through JSON slice files
slice_dir = '/json_slices_master_dataset'  # Directory containing the JSON slice files
num_slices = 1000  # Number of slices; adjust this as needed

slice_files = [f'mpd.slice.{i}-{i+999}.json' for i in range(0, num_slices*1000, 1000)]

In [9]:
# Step 2: Process JSON File in Chunks
chunk_size = 1000
chunk = []
chunk_count = 0

for slice_file in slice_files:
    slice_path = os.path.join(slice_dir, slice_file)
    chunk = []

    with open(slice_path, 'r') as file:
        for playlist in ijson.items(file, 'playlists.item'):
            chunk.append(playlist)
            if len(chunk) >= chunk_size:
                # Process the chunk
                filtered_playlists = process_chunk(chunk)
                # Save intermediate result
                with open(f'filtered_playlists_{chunk_count}.json', 'w') as outfile:
                    json.dump({"playlists": filtered_playlists}, outfile, indent=4)
                chunk = []
                chunk_count += 1


FileNotFoundError: [Errno 2] No such file or directory: '/json_slices_master_dataset\\mpd.slice.0-999.json'

In [ ]:
# Process any remaining items in the last chunk
if chunk:
        filtered_playlists = process_chunk(chunk)
        with open(f'filtered_playlists_{chunk_count}.json', 'w') as outfile:
            json.dump({"playlists": filtered_playlists}, outfile, indent=4)
        chunk = []
        chunk_count += 1

In [ ]:
# Step 3: Combine Intermediate Results into a Final Dataset
final_playlists = []

for i in range(chunk_count):
    with open(f'filtered_playlists_{i}.json', 'r') as infile:
        data = json.load(infile)
        final_playlists.extend(data['playlists'])

with open('final_filtered_playlists.json', 'w') as final_file:
    json.dump({"playlists": final_playlists}, final_file, indent=4)